**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from collections import Counter
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups
from sklearn.neighbors import KDTree

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split, KFold, cross_val_score
# from sklearn.cross_validation import KFold
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [2]:
# %%pycodestyle


class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute', p=2):
        self.p = p
        self.n_neighbors = n_neighbors
        self.alg_type = algorithm

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        if self.alg_type == 'brute':
            distances = self.calc_dist(X)
            return self.predict_labels(distances)
        elif self.alg_type == 'kd_tree':
            self.tree = KDTree(self.X_train)
            return self.predict_labels_kd(X)
        
    def predict_labels_kd(self, X):
        y_pred = np.zeros(X.shape[0])
#         print(self.X_train)
        for i in range(X.shape[0]):
#             print(X[i,:])
            dists, k_closest = self.tree.query(X[i,:][np.newaxis,:], k=self.n_neighbors)
            neighbor_labels = self.y_train[k_closest[0]]
#             print(type(neighbor_labels))
#             print(neighbor_labels)
            dic = Counter(neighbor_labels)
#             print(dic)
            y_pred[i] = dic.most_common(1)[0][0]
        return y_pred

    def predict_labels(self, dists):
        y_pred = np.ones(dists.shape[0])
        for i in range(dists.shape[0]):
            k_closest_ind = np.argsort(dists[i])[:self.n_neighbors]
            neighbors_labels = self.y_train[k_closest_ind]
            counter_dict = Counter(neighbors_labels)
            y_pred[i] = counter_dict.most_common(1)[0][0]
        return y_pred

    def predict_proba(self, X):
        probs = np.zeros((X.shape[0], np.unique(self.y_train).shape[0]))
        if self.alg_type == 'brute':
            dists = self.calc_dist(X)
#         print(X.shape[0])
#         print(np.unique(self.y_train).shape[0])
#             probs = np.zeros((X.shape[0], np.unique(self.y_train).shape[0]))
            for i in range(dists.shape[0]):
                k_closest_ind = np.argsort(dists[i])[:self.n_neighbors]
                neighbors_labels = self.y_train[k_closest_ind]
                dic = Counter(neighbors_labels)
                for j in range(probs.shape[1]):
                    probs[i,j] = dic[j] / neighbors_labels.shape[0]
#             return probs
        if self.alg_type == 'kd_tree':
            for i in range(X.shape[0]):
                dists, k_closest = self.tree.query(X[i,:][np.newaxis,:], k=self.n_neighbors)
                neighbor_labels = self.y_train[k_closest[0]]
                dic = Counter(neighbor_labels)
                for j in range(probs.shape[1]):
                    probs[i,j] = dic[j] / neighbor_labels.shape[0]
        return probs
            
    
    def calc_dist(self, X):
        dists = np.zeros((X.shape[0], self.X_train.shape[0]))
        power = np.ones(self.X_train.shape)
        power[:] = self.p
        rev_power = (np.ones(self.X_train.shape[0]) / self.p)
        for i in range(X.shape[0]):
            dists[i, :] = np.float_power(np.sum(np.float_power(self.X_train-X[i, :], power), axis=1), rev_power)
        return dists
                         
    def score(self, X, y):
        return np.mean(self.predict(X) == y)

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [ ]:
print(iris.data)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [6]:
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [7]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [8]:
my_clf.score(X_test, y_test)

0.9333333333333333

In [9]:
clf.score(X_test, y_test)

0.9333333333333333

In [10]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [11]:
%time clf.fit(X_train, y_train)

CPU times: user 909 µs, sys: 213 µs, total: 1.12 ms
Wall time: 1.02 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [12]:
%time my_clf.fit(X_train, y_train)

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11 µs


In [13]:
%time clf.predict(X_test)

CPU times: user 2.49 ms, sys: 898 µs, total: 3.38 ms
Wall time: 2.57 ms


array([1, 1, 0, 1, 0, 2, 0, 1, 1, 1, 0, 0, 2, 2, 2])

In [14]:
%time my_clf.predict(X_test)

CPU times: user 2.07 ms, sys: 586 µs, total: 2.66 ms
Wall time: 2.03 ms


array([1., 1., 0., 1., 0., 2., 0., 1., 1., 1., 0., 0., 2., 2., 2.])

In [15]:
%time clf.predict_proba(X_test)

CPU times: user 1.33 ms, sys: 654 µs, total: 1.99 ms
Wall time: 1.39 ms


array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [16]:
%time my_clf.predict_proba(X_test)

CPU times: user 1.96 ms, sys: 880 µs, total: 2.84 ms
Wall time: 1.96 ms


array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [17]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [18]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [19]:
%time clf.fit(X_train, y_train)

CPU times: user 1.15 ms, sys: 937 µs, total: 2.08 ms
Wall time: 6.19 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [20]:
%time my_clf.fit(X_train, y_train)

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11.2 µs


In [21]:
%time clf.predict(X_test)

CPU times: user 2.91 ms, sys: 2.4 ms, total: 5.31 ms
Wall time: 5.26 ms


array([0, 0, 1, 1, 1, 2, 1, 0, 2, 1, 2, 2, 0, 2, 0])

In [22]:
%time my_clf.predict(X_test)

CPU times: user 2.47 ms, sys: 232 µs, total: 2.7 ms
Wall time: 2.56 ms


array([0., 0., 1., 1., 1., 2., 1., 0., 2., 1., 2., 2., 0., 2., 0.])

In [23]:
%time clf.predict_proba(X_test)

CPU times: user 1.09 ms, sys: 262 µs, total: 1.35 ms
Wall time: 1.11 ms


array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [24]:
%time my_clf.predict_proba(X_test)

CPU times: user 2.81 ms, sys: 742 µs, total: 3.55 ms
Wall time: 2.83 ms


array([[1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [25]:
my_clf.score(X_test, y_test)

1.0

In [26]:
clf.score(X_test, y_test)

1.0

In [27]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [28]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [29]:
data = newsgroups['data']
target = newsgroups['target']

Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [30]:
data_tok = list()
target_list = list()
for i in range(len(data)):
    new_data = ''
    for j in range(len(data[i])):
#         print(new_data)
        if (data[i][j].isalpha() or data[i][j].isalnum()):
            new_data += data[i][j].lower()
        else:
            new_data += ' '
    new_data = new_data.split(' ')
    try:
        while True:
            new_data.remove('')
    except ValueError:
        pass
    if new_data != list():
        data_tok.append(new_data)
        target_list.append(target[i])
        

In [31]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

In [32]:
main_dic = {}
for i in range(len(data_tok)):
    cnt = Counter(data_tok[i])
    main_dic[i] = cnt
# main_dic[1]

In [33]:
import scipy.sparse

data = []
row = []
col = []

uniq_dic = {}
counter = 0

for i in range(len(main_dic)):
    for word in main_dic[i].keys():
        row.append(i)
        if word not in uniq_dic.keys():
            uniq_dic[word] = counter
            data.append(main_dic[i][word])
            counter += 1
        else:
            data.append(0)
        col.append(uniq_dic[word])
        
print(len(main_dic))
col_t = [0 for i in range(len(target_list))]
row_t = list(range(len(target_list)))
matrix_data = scipy.sparse.csr_matrix((data, (row, col)), dtype=np.int).toarray()
matrix_target = np.array(target_list, dtype=np.int)
# matrix_target = scipy.sparse.csr_matrix((target_list, (row_t, col_t)), dtype=np.int).toarray()

# matrix_data
print(matrix_target)

11004
[7 4 4 ... 3 1 8]


In [34]:
print(matrix_data)

[[3 4 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 1 1]]


*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [ ]:
kf = KFold(n_splits=3)
X_tr_list = list()
X_ts_list = list()
y_tr_list = list()
y_ts_list = list()

for train_ind, test_ind in kf.split(iris.data):
#     print(train_ind)
#     print(test_ind)
    X_train, X_test = iris.data[train_ind], iris.data[test_ind]
    y_train, y_test = iris.target[train_ind], iris.target[test_ind]
    X_tr_list.append(X_train)
    X_ts_list.append(X_test)
    y_tr_list.append(y_train)
    y_ts_list.append(y_test)

# print(X_tr_list[0])
# print(y_tr_list[0])

In [ ]:
print(type(X_tr_list[0]))

In [ ]:
print(y_tr_list[0])

In [ ]:
print(y_tr_list[1])

In [ ]:
def mean_list(lst):
    sum = 0
    for i in range(len(lst)):
        sum += lst[i]
    return sum / len(lst)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(matrix_data, matrix_target, test_size=0.33, stratify=matrix_target)
clf1 = KNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)
mass = cross_val_score(clf, iris.data, iris.target, cv=3, n_jobs=-1)

In [ ]:
mass

In [ ]:
clf1 = MyKNeighborsClassifier(algorithm='kd_tree', n_neighbors=2)
clf1.fit(X_tr_list[0],)

In [ ]:
mass

In [ ]:
score_list = list()
for neighbor_am in range(1,11):
    clf1 = KNeighborsClassifier(algorithm='kd_tree', n_neighbors=neighbor_am)
#     m = list()
    print(neighbor_am)
#     for j in range(3):
#         rng = X_ts_list[j].shape[0]
    m = cross_val_score(clf1, matrix_data, matrix_target, cv=3, n_jobs=-1)
#         clf1.fit(X_tr_list[j][:rng:,:], y_tr_list[j][:rng:])
#         m.append(clf1.score(X_ts_list[j][:rng:,:], y_ts_list[j][:rng:]))
#         print(m[len(m)-1])
    print(mean_list(m))
    score_list.append(mean_list(m))

tmp = score_list.copy()
tmp.sort()
best_amount = score_list.index(tmp[len(tmp)-1]) + 1
print(score_list)
print(best_amount)

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.